In [9]:
%pip install git+https://github.com/bigscience-workshop/petals
%pip install --upgrade langchain

  Cloning https://github.com/bigscience-workshop/petals to /tmp/pip-req-build-qg6j88eg
  Running command git clone --filter=blob:none --quiet https://github.com/bigscience-workshop/petals /tmp/pip-req-build-qg6j88eg
  Resolved https://github.com/bigscience-workshop/petals to commit 82a97d6e9ea18e79639f375f124c4c83fe1933e8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import AutoTokenizer
from petals import AutoDistributedModelForCausalLM
from langchain.llms.base import LLM

In [11]:
class CustomLLM(LLM):
    def __init__(self, model_name: str,max_tokens: int = 15, **kwargs):
        super().__init__(**kwargs)
        tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, add_bos_token=False)
        model = AutoDistributedModelForCausalLM.from_pretrained(model_name)
        model = model.cuda()  # Move the model to GPU if available
        object.__setattr__(self, "tokenizer", tokenizer)
        object.__setattr__(self, "model", model)
        object.__setattr__(self, "max_tokens", max_tokens)

    def _call(self, text: str, stop=None, **kwargs) -> str:
        inputs = self.tokenizer(text, return_tensors="pt")["input_ids"].cuda()
        outputs = self.model.generate(inputs, max_new_tokens=self.max_tokens)
        decoded_output = (self.tokenizer.decode(outputs[0]))
        return decoded_output

    @property
    def _llm_type(self) -> str:
        return "CustomLLM"

In [12]:
llm = CustomLLM(model_name="petals-team/StableBeluga2")

Nov 02 01:34:10.268 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Nov 02 01:34:10.269 [INFO] Using DHT prefix: StableBeluga2-hf
torch_shm_manager: error while loading shared libraries: libcupti.so.12: cannot open shared object file: No such file or directory


RuntimeError: no response from torch_shm_manager at "/home/biscuitbobby/Documents/Projects/semantic_kernel_test/venv/lib64/python3.11/site-packages/torch/bin/torch_shm_manager"

In [ ]:
# Define the prompt template
template = """
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
# Create the LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)


In [ ]:
# Generate a response
question = input("query: ")
response = llm_chain.run(question)
print(response)
